##### Copyright 2020 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# feels.like with TensorFlow Lite Model Maker with TensorFlow 2.0

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/examples/blob/master/lite/codelabs/flower_classification/ml/Flower_Classification_with_TFLite_Model_Maker.ipynb">      
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/examples/blob/master/lite/codelabs/flower_classification/ml/Flower_Classification_with_TFLite_Model_Maker.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
</table>

Model Maker library simplifies the process of adapting and converting a TensorFlow neural-network model to particular input data when deploying this model for on-device ML applications.

This notebook shows an end-to-end example that utilizes this Model Maker library to illustrate the adaption and conversion of a commonly-used image classification model to classify flowers on a mobile device.

# Additional Data Sources

DeepFashion (http://mmlab.ie.cuhk.edu.hk/projects/DeepFashion.html)

@inproceedings{liuLQWTcvpr16DeepFashion,
 author = {Liu, Ziwei and Luo, Ping and Qiu, Shi and Wang, Xiaogang and Tang, Xiaoou},
 title = {DeepFashion: Powering Robust Clothes Recognition and Retrieval with Rich Annotations},
 booktitle = {Proceedings of IEEE Conference on Computer Vision and Pattern Recognition (CVPR)},
 month = {June},
 year = {2016} 
 }

## Prerequisites

To run this example, we first need to install serveral required packages, including Model Maker package that in github [repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).

In [2]:
!pip install tflite-model-maker-nightly

     |████████████████████████████████| 593kB 9.1MB/s 
     |████████████████████████████████| 1.1MB 19.0MB/s 
     |████████████████████████████████| 1.2MB 51.7MB/s 
     |████████████████████████████████| 112kB 54.9MB/s 
     |████████████████████████████████| 174kB 46.1MB/s 
     |████████████████████████████████| 1.1MB 51.0MB/s 
     |████████████████████████████████| 849kB 49.8MB/s 
     |████████████████████████████████| 6.3MB 27.8MB/s 
     |████████████████████████████████| 645kB 47.1MB/s 
     |████████████████████████████████| 92kB 12.5MB/s 
     |████████████████████████████████| 122kB 48.8MB/s 
     |████████████████████████████████| 194kB 53.2MB/s 
     |████████████████████████████████| 38.2MB 64kB/s 
     |████████████████████████████████| 102kB 12.7MB/s 
     |████████████████████████████████| 686kB 36.4MB/s 
     |████████████████████████████████| 358kB 50.6MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115928 sha256=13e5fe0bdbf69eb121a76

Import the required packages.

In [3]:
import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.image_classifier import DataLoader

## Simple End-to-End Example

### Get the data path

In [4]:
import os
os.getcwd()

'/content'

In [5]:
import shutil
shutil.unpack_archive("fashionimgs.zip")
image_path = os.path.join(os.path.dirname('fashionimgs.zip'), 'fashionimgs')

### Run the example
The example just consists of 4 lines of code as shown below, each of which representing one step of the overall process.


1.   Load input data specific to an on-device ML app. Split it to training data and testing data.

In [6]:
data = DataLoader.from_folder(image_path)
#90/5/5 split
train_data, rest_data = data.split(0.9)
validationdata, test_data = rest_data.split(0.5)

INFO:tensorflow:Load image with size: 2560, num_label: 13, labels: 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90.


2. Customize the TensorFlow model.

In [9]:
#define hyperparameters
epoch = 15
dropout = [0, 0.2, 0.3]

In [10]:
for i in dropout:
  print(i)
  model = image_classifier.create(train_data, model_spec=model_spec.get('efficientnet_lite4'), epochs=epoch, dropout_rate=i, validation_data=validationdata, shuffle=True)
  loss, accuracy = model.evaluate(test_data)
  print('#########################')

0
INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2_1 (HubK (None, 1280)              11837936  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 13)                16653     
Total params: 11,854,589
Trainable params: 16,653
Non-trainable params: 11,837,936
_________________________________________________________________
None


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/15
72/72 [==============================] - 19s 212ms/step - loss: 2.2116 - accuracy: 0.2973 - val_loss: 1.9923 - val_accuracy: 0.3516
Epoch 2/15
72/72 [==============================] - 15s 207ms/step - loss: 1.8723 - accuracy: 0.4132 - val_loss: 1.9001 - val_accuracy: 0.3984
Epoch 3/15
72/72 [==============================] - 15s 207ms/step - loss: 1.7939 - accuracy: 0.4523 - val_loss: 1.8667 - val_accuracy: 0.4062
Epoch 4/15
72/72 [==============================] - 15s 205ms/step - loss: 1.7529 - accuracy: 0.4683 - val_loss: 1.8612 - val_accuracy: 0.3906
Epoch 5/15
72/72 [==============================] - 16s 218ms/step - loss: 1.7203 - accuracy: 0.4891 - val_loss: 1.8355 - val_accuracy: 0.3984
Epoch 6/15
72/72 [==============================] - 15s 203ms/step - loss: 1.6834 - accuracy: 0.5087 - val_loss: 1.8286 - val_accuracy: 0.3828
Epoch 7/15
72/72 [==============================] - 15s 204ms/step - loss: 1.6614 - accuracy: 0.5148 - val_loss: 1.8071 - val_accuracy: 0.4297

INFO:tensorflow:Retraining the models...


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2_2 (HubK (None, 1280)              11837936  
_________________________________________________________________
dropout_2 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 13)                16653     
Total params: 11,854,589
Trainable params: 16,653
Non-trainable params: 11,837,936
_________________________________________________________________
None
Epoch 1/15
72/72 [==============================] - 19s 213ms/step - loss: 2.2422 - accuracy: 0.2826 - val_loss: 1.9921 - val_accuracy: 0.3594
Epoch 2/15
72/72 [==============================] - 15s 207ms/step - loss: 1.9119 - accuracy: 0.4084 - val_loss: 1.8956 - val_accuracy: 0.3906
Epoch 3/15
72/72 [==============================] - 15s 207ms/step - loss: 1

INFO:tensorflow:Retraining the models...


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2_3 (HubK (None, 1280)              11837936  
_________________________________________________________________
dropout_3 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 13)                16653     
Total params: 11,854,589
Trainable params: 16,653
Non-trainable params: 11,837,936
_________________________________________________________________
None
Epoch 1/15
72/72 [==============================] - 18s 213ms/step - loss: 2.2365 - accuracy: 0.2882 - val_loss: 1.9531 - val_accuracy: 0.3828
Epoch 2/15
72/72 [==============================] - 15s 208ms/step - loss: 1.9179 - accuracy: 0.4045 - val_loss: 1.8935 - val_accuracy: 0.3984
Epoch 3/15
72/72 [==============================] - 15s 207ms/step - loss: 1

3. Evaluate the model.

In [ ]:
loss, accuracy = model.evaluate(test_data)

1/1 [==============================] - 1s 639ms/step - loss: 2.1045 - accuracy: 0.3333


4.  Export to TensorFlow Lite model.
You could download it in the left sidebar same as the uploading part for your own use.

In [ ]:
model.export(export_dir='.', with_metadata=False)

After this simple 4 steps, we can now download the model and label files, and continue to the next step in the [codelab](https://codelabs.developers.google.com/codelabs/recognize-flowers-with-tensorflow-on-android/#4).

For a more comprehensive guide to TFLite Model Maker, please refer to this [notebook](https://colab.sandbox.google.com/github/tensorflow/examples/blob/master/tensorflow_examples/lite/model_maker/demo/image_classification.ipynb) and its [documentation](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).
